In [13]:
import sys
sys.path.append("..")

# Reload modules in /src/ when changed
%load_ext autoreload
%autoreload 2

from fxml.data.datasets.direction_dataset import DirectionDataset

import pandas as pd
import numpy as np

In [20]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1

START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"

In [21]:
DIRECTION_LABEL_FILE_PATH = '../data/labels/direction_labels/USDJPY-1m-20210101-20241231-CUSUM_0.0004-TB.pkl'

In [22]:
from torch.utils.data import Dataset, DataLoader

In [23]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
labels = pd.read_pickle(DIRECTION_LABEL_FILE_PATH)

In [24]:
print(df.shape)
df.head()

(1486378, 67)


,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-04 13:33:00,102.9910,102.9960,102.9875,102.9930,2.465100e+11,0.002329,0.000015,0.0015,0.000015,1.920436,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:34:00,102.9940,102.9960,102.9745,102.9840,1.893000e+11,0.002426,-0.000087,-0.0090,-0.000087,1.909965,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:35:00,102.9850,102.9850,102.9670,102.9720,4.723400e+11,0.002645,-0.000117,-0.0120,-0.000117,1.901790,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:36:00,102.9710,102.9735,102.9495,102.9525,4.980100e+11,0.002354,-0.000189,-0.0195,-0.000189,1.888939,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 13:37:00,102.9535,102.9570,102.9480,102.9530,2.309000e+11,0.002556,0.000005,0.0005,0.000005,1.900871,...,4,1,-0.258819,-0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025


In [25]:
print(labels.shape)
labels.head()

(98766, 5)


,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2021-01-04 14:00:00,2021-01-04 14:02:00,0.000183,0.000228,1.0,2.0
2021-01-04 14:21:00,2021-01-04 14:24:00,0.000167,-0.000214,-1.0,0.0
2021-01-04 14:35:00,2021-01-04 14:41:00,0.000176,-0.000199,-1.0,0.0
2021-01-04 14:51:00,2021-01-04 15:05:00,0.000166,-0.000223,-1.0,0.0
2021-01-04 15:07:00,2021-01-04 15:19:00,0.000197,0.000267,1.0,2.0
